In [112]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [113]:
df2020 = pd.read_csv('2020.csv')

In [114]:
df2020.head()

,player_id,adjusted_net_yards_per_attempt_index,adjusted_net_yards_per_pass_attempt,adjusted_yards_per_attempt,adjusted_yards_per_attempt_index,all_purpose_yards,approximate_value,assists_on_tackles,attempted_passes,birth_date,...,yards_per_completed_pass,yards_per_game_played,yards_per_kickoff_return,yards_per_punt,yards_per_punt_return,yards_per_touch,yards_recovered_from_fumble,yards_returned_from_interception,year,id
0,AdamDa01,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,1992-12-24,...,NaN,NaN,NaN,NaN,NaN,11.9,NaN,NaN,2020,AdamDa01 2020
1,AdamMo00,NaN,NaN,NaN,NaN,NaN,1.0,5.0,NaN,1995-07-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,AdamMo00 2020
2,AlexJa00,NaN,NaN,NaN,NaN,NaN,11.0,11.0,NaN,1997-02-09,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-4.0,2020,AlexJa00 2020
3,AmosAd00,NaN,NaN,NaN,NaN,NaN,6.0,18.0,NaN,1993-04-29,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,47.0,2020,AmosAd00 2020
4,AustTa00,NaN,NaN,NaN,NaN,34.0,0.0,NaN,NaN,1990-03-15,...,NaN,NaN,NaN,NaN,4.7,4.0,0.0,NaN,2020,AustTa00 2020


In [115]:
df2020 = df2020[df2020['team_abbreviation'] != '2TM']
df2020 = df2020[df2020['team_abbreviation'] != '3TM']
# dropping duplicates based on id as those are unique values
df2020 = df2020.drop_duplicates(subset='id')

In [116]:
rushing_columns = ['rush_attempts','rush_yards','rush_yards_per_attempt','rush_touchdowns']
receiving_columns = ['times_pass_target','receptions','receiving_yards','receiving_yards_per_reception','receiving_touchdowns']
passing_columns = ['attempted_passes','passing_yards','passing_touchdowns','interceptions_thrown']

In [117]:
def transform_columns(df,new_column_list):
    df = df[['name','team_abbreviation','games'] + new_column_list + ['fumbles']]
    return df

In [118]:
rb_df = transform_columns(rb_df,rushing_columns+receiving_columns)
wr_df = transform_columns(wr_df,rushing_columns+receiving_columns)
te_df = transform_columns(te_df,receiving_columns)
qb_df = transform_columns(qb_df,passing_columns)

In [119]:
rb_df.head()

,name,team_abbreviation,games,rush_attempts,rush_yards,rush_yards_per_attempt,rush_touchdowns,times_pass_target,receptions,receiving_yards,receiving_yards_per_reception,receiving_touchdowns,fumbles
316,Alvin Kamara,NOR,15.0,187.0,932.0,5.0,16.0,107.0,83.0,756.0,9.1,5.0,1.0
709,Dalvin Cook,MIN,14.0,312.0,1557.0,5.0,16.0,54.0,44.0,361.0,8.2,1.0,5.0
232,Derrick Henry,TEN,16.0,378.0,2027.0,5.4,17.0,31.0,19.0,114.0,6.0,0.0,3.0
29,Aaron Jones,GNB,14.0,201.0,1104.0,5.5,9.0,63.0,47.0,355.0,7.6,2.0,2.0
1545,David Montgomery,CHI,15.0,247.0,1070.0,4.3,8.0,68.0,54.0,438.0,8.1,2.0,1.0


In [120]:
#Create new column to calculate fantasy points scored (Note: this is full PPR)
rb_df['FantasyPoints'] = (rb_df['rush_yards']*0.1 + rb_df['rush_touchdowns']*6 + rb_df['receptions'] 
                          + rb_df['receiving_yards']*0.1 + rb_df ['receiving_touchdowns']*6 - rb_df['fumbles']*2)

#Create new column for Fantasy points per game.
rb_df['FantasyPoints/GM'] = rb_df['FantasyPoints']/rb_df['games']
rb_df['FantasyPoints/GM'] = rb_df['FantasyPoints/GM'].apply(lambda x: round(x, 2))

#Create new column for usage per game. Usage is defined as # of targets + carries
rb_df['Usage/GM'] = (rb_df['rush_attempts'] + rb_df['times_pass_target'])/rb_df['games']
#round each row value to two decimal places
rb_df['Usage/GM'] = rb_df['Usage/GM'].apply(lambda x: round(x, 2))
rb_df = rb_df.sort_values(by=['FantasyPoints/GM'],ascending=False)

In [121]:
#Create new column to calculate fantasy points scored (Note: this is full PPR)
wr_df['FantasyPoints'] = (wr_df['rush_yards']*0.1 + wr_df['rush_touchdowns']*6 + wr_df['receptions'] 
                          + wr_df['receiving_yards']*0.1 + wr_df ['receiving_touchdowns']*6 - wr_df['fumbles']*2)

#Create new column for Fantasy points per game.
wr_df['FantasyPoints/GM'] = wr_df['FantasyPoints']/wr_df['games']
wr_df['FantasyPoints/GM'] = wr_df['FantasyPoints/GM'].apply(lambda x: round(x, 2))

#Create new column for usage per game. Usage is defined as # of targets + carries
wr_df['Usage/GM'] = (wr_df['rush_attempts'] + wr_df['times_pass_target'])/wr_df['games']
#round each row value to two decimal places
wr_df['Usage/GM'] = wr_df['Usage/GM'].apply(lambda x: round(x, 2))
wr_df = wr_df.sort_values(by=['FantasyPoints/GM'],ascending=False)

In [122]:
#Create new column to calculate fantasy points scored (Note: this is full PPR)
te_df['FantasyPoints'] = (te_df['receptions'] 
                          + te_df['receiving_yards']*0.1 + te_df['receiving_touchdowns']*6 - te_df['fumbles']*2)

#Create new column for Fantasy points per game.
te_df['FantasyPoints/GM'] = te_df['FantasyPoints']/te_df['games']
te_df['FantasyPoints/GM'] = te_df['FantasyPoints/GM'].apply(lambda x: round(x, 2))

#Create new column for usage per game. Usage is defined as # of targets + carries
te_df['Usage/GM'] = (te_df['times_pass_target'])/te_df['games']
#round each row value to two decimal places
te_df['Usage/GM'] = te_df['Usage/GM'].apply(lambda x: round(x, 2))
te_df = te_df.sort_values(by=['FantasyPoints/GM'],ascending=False)

In [123]:
#Create new column to calculate fantasy points scored (Note: this is full PPR)
qb_df['FantasyPoints'] = (qb_df['passing_touchdowns']*4 + (qb_df['passing_yards']/25) - qb_df['interceptions_thrown']*(-2))

#Create new column for Fantasy points per game.
qb_df['FantasyPoints/GM'] = qb_df['FantasyPoints']/qb_df['games']
qb_df['FantasyPoints/GM'] = qb_df['FantasyPoints/GM'].apply(lambda x: round(x, 2))

#Create new column for usage per game. Usage is defined as # of targets + carries
qb_df['Usage/GM'] = (qb_df['attempted_passes']/qb_df['games'])
#round each row value to two decimal places
qb_df['Usage/GM'] = qb_df['Usage/GM'].apply(lambda x: round(x, 2))
qb_df = qb_df.sort_values(by=['FantasyPoints/GM'],ascending=False)

In [124]:
rb_df.head()

,name,team_abbreviation,games,rush_attempts,rush_yards,rush_yards_per_attempt,rush_touchdowns,times_pass_target,receptions,receiving_yards,receiving_yards_per_reception,receiving_touchdowns,fumbles,FantasyPoints,FantasyPoints/GM,Usage/GM
316,Alvin Kamara,NOR,15.0,187.0,932.0,5.0,16.0,107.0,83.0,756.0,9.1,5.0,1.0,375.8,25.05,19.60
709,Dalvin Cook,MIN,14.0,312.0,1557.0,5.0,16.0,54.0,44.0,361.0,8.2,1.0,5.0,327.8,23.41,26.14
232,Derrick Henry,TEN,16.0,378.0,2027.0,5.4,17.0,31.0,19.0,114.0,6.0,0.0,3.0,329.1,20.57,25.56
29,Aaron Jones,GNB,14.0,201.0,1104.0,5.5,9.0,63.0,47.0,355.0,7.6,2.0,2.0,254.9,18.21,18.86
1545,David Montgomery,CHI,15.0,247.0,1070.0,4.3,8.0,68.0,54.0,438.0,8.1,2.0,1.0,262.8,17.52,21.00


In [125]:
rb_df['Diff'] = rb_df['FantasyPoints/GM'].diff()
wr_df['Diff'] = wr_df['FantasyPoints/GM'].diff()
qb_df['Diff'] = qb_df['FantasyPoints/GM'].diff()
te_df['Diff'] = te_df['FantasyPoints/GM'].diff()
rb_df.head()

,name,team_abbreviation,games,rush_attempts,rush_yards,rush_yards_per_attempt,rush_touchdowns,times_pass_target,receptions,receiving_yards,receiving_yards_per_reception,receiving_touchdowns,fumbles,FantasyPoints,FantasyPoints/GM,Usage/GM,Diff
316,Alvin Kamara,NOR,15.0,187.0,932.0,5.0,16.0,107.0,83.0,756.0,9.1,5.0,1.0,375.8,25.05,19.60,NaN
709,Dalvin Cook,MIN,14.0,312.0,1557.0,5.0,16.0,54.0,44.0,361.0,8.2,1.0,5.0,327.8,23.41,26.14,-1.64
232,Derrick Henry,TEN,16.0,378.0,2027.0,5.4,17.0,31.0,19.0,114.0,6.0,0.0,3.0,329.1,20.57,25.56,-2.84
29,Aaron Jones,GNB,14.0,201.0,1104.0,5.5,9.0,63.0,47.0,355.0,7.6,2.0,2.0,254.9,18.21,18.86,-2.36
1545,David Montgomery,CHI,15.0,247.0,1070.0,4.3,8.0,68.0,54.0,438.0,8.1,2.0,1.0,262.8,17.52,21.00,-0.69
